In [81]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn import tree
from sklearn.naive_bayes import GaussianNB

import time

from skimage.feature import hog

# Lee los datos y guárdalos en data
data = np.load("dataset_detection/train_signs_detection.npy")
Xtrain = data[:, :-1]
ytrain = data[:, -1].reshape(-1, 1)
print(Xtrain.shape)

rn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
rn.fit(Xtrain, ytrain)

NB = GaussianNB()
NB.fit(Xtrain, ytrain)

(1584, 8100)


C:\Users\Arribas\anaconda3\envs\py37aprendizaje\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\Arribas\anaconda3\envs\py37aprendizaje\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GaussianNB()

In [89]:
import imutils
import argparse
import cv2
import time

def pyramid(image, scale=1.5, min_size=(30, 30)):
    """
    Genera imágenes sucesivamente más pequeñas con base a los parámetros proporcionados.
    :param image: Imagen base.
    :param scale: Factor de reducción de cada nivel de la pirámide.
    :param min_size: Dimensiones mínimas que debe tener un nivel de la pirámide.
    :return: Un nuevo nivel de la pirámide en cada llamada.
    """
    yield image  # El primer de la pirámide será la imagen original.

    while True:
        # Reducimos el ancho de la imagen con base a `scale`, y luego la
        # redimensionamos manteniendo la relación de aspecto.
        width = int(image.shape[1] / scale)
        image = imutils.resize(image, width=width)

        # Si el resultado es una imagen más pequeña de lo esperada, paramos.
        if image.shape[0] < min_size[1] or image.shape[1] < min_size[0]:
            break

        # Retorna el siguiente nivel de la pirámide.
        yield image


def sliding_window(image, step_size, window_size):
    """
    Genera ventanas de tamaño `window_size` hasta haber recorrido toda la imagen.
    :param image: Imagen sobre la cual generar ventanas.
    :param step_size: Número de píxeles que nos moveremos en cada dirección.
    :param window_size: Dimensiones de la ventana deslizante.
    :return: Coordenadas (x, y) de la esquina superior izquierda de la ventana, y la región como tal.
    """
    for x in range(0, image.shape[0], step_size):
        for y in range(0, image.shape[1], step_size):
            yield x, y, image[x:x + window_size[1], y:y + window_size[0]]

In [14]:
# #Test de la pirámide de imágenes

# # Leemos la imagen.
# image = cv2.imread('Dataset_images/00001.jpg',0)

# # Generamos y enumeramos cada nivel de la pirámide.
# for index, level in enumerate(pyramid(image, scale=1.5)):
#     # Mostramos el nivel actual.
#     cv2.imshow(f'Nivel {index + 1}', level)
#     print(f'Dimensiones: {level.shape}')
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

In [88]:
#Test de la ventana deslizante
nAcierto = 0
# Cargamos la imagen en memoria.
rutaImg = 'Dataset_images/00001.jpg'
image = cv2.imread(rutaImg, 0)
imagenColor = cv2.imread(rutaImg)
#Array de ventanas que han detectado una señal
detections = []
k = 1
tam = 100
#Iteramos cada nivel/imagen de la pirámide
for index, layer in enumerate(pyramid(image, scale=1.5)):
    print('Pyramid layer {}'.format(index))
    # ... y en cada uno deslizaremos una ventana con las proporciones especificadas, moviéndonos 32 píxeles a la vez.
    for x, y, window in sliding_window(layer, step_size=32, window_size=(tam, tam)):
        # Si la ventana actual no cumple con el tamaño deseado, la ignoramos.
        
        #Hemos añadido una condicion para evitar que prediga fotos de cielo q daban errores
        if window.shape == (tam,tam) and np.mean(window) < 230:
#             window = cv2.resize(window,(100,100))
            acc = clf.predict_proba(hog(cv2.equalizeHist(window)).reshape(1,-1))

            if acc[0,1] > 0.9:       
                clone = layer.copy()
                print('Coordenadas ventana recorte: {} {}'.format(x,y))
                print('Acuraccy',acc)
#                 cv2.imshow(str(x)+' ', window)
                nAcierto += 1
                #Guardar % comienzo x, % comienzo y, % píxeles que ocupa la ventana y acc.
                detections.append((x/layer.shape[0], y/layer.shape[1], 100/layer.shape[0], acc[0,1]))

                print('Coordenadas ventana imagen grande: {} {}'.format(x,y))
                cv2.rectangle(clone, (y,x), (y + 100, x + 100), (0, 128, 0), 2)
                #cv2.rectangle(clone, (416,64), (416 + 100, 64 + 100), (0, 128, 0), 2)
                label = "{}: {:.2f}%".format('Sign', acc[0,1] * 100)

                label2 = 'Coordenadas -> x: ' + str(y) + ' y: ' + str(x)
                y2 = y - 15 if y - 15 > 15 else y + 15
                cv2.putText(clone, label, (y2, x), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
                cv2.putText(clone, label2, (150, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
                #k = k + 1
            
detections = np.array(detections)     
clone = imagenColor.copy()

for i in range(0,detections.shape[0]):
    x = int(detections[i,0]*clone.shape[0]) #Consigo el punto donde comienza la ventana
    y = int(detections[i,1]*clone.shape[1])
    tamVentana = int(detections[i,2]*clone.shape[0])
    precision = detections[i,3]
    
    #clone = layer.copy()
    cv2.rectangle(clone, (y,x), (y + tamVentana, x + tamVentana), (0, 128, 0), 2)
    label = "{}: {:.2f}%".format('Sign', precision * 100)
    y2 = y - 20 if y - 20 > 15 else y + 15
    cv2.putText(clone, label, (y2, x), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,128,0), 2)
    
#cv2.imwrite('copiaresultadofinal.jpg', image)

cv2.imshow('Ventana', clone) 
cv2.waitKey(0)
cv2.destroyAllWindows()
for i in detections:
    print(i)

Pyramid layer 0
Coordenadas ventana recorte: 416 64
Acuraccy [[0.03238849 0.96761151]]
Coordenadas ventana imagen grande: 416 64
Pyramid layer 1
Pyramid layer 2
Pyramid layer 3
Pyramid layer 4
Pyramid layer 5
Pyramid layer 6
Pyramid layer 7
[0.52       0.04705882 0.125      0.96761151]
